In [16]:
import numpy as np
import scipy.io.wavfile as wav
from pydub import AudioSegment
import os

def compress_audio_simple(input_audio_file, output_audio_file, k):
    # Convertir en mono si nécessaire
    audio = AudioSegment.from_file(input_audio_file)
    if audio.channels > 1:
        print("Conversion du fichier audio en mono...")
        audio = audio.set_channels(1)
        audio.export("temp_mono.wav", format="wav")
        input_audio_file = "temp_mono.wav"

    # Charger le fichier audio (désormais mono)
    sample_rate, audio_data = wav.read(input_audio_file)
    
    # Si audio_data est 1D, le transformer en 2D
    if audio_data.ndim == 1:
        print("Conversion des données audio en matrice 2D...")
        audio_data = audio_data.reshape(-1, 1)
    
    # Appliquer la décomposition SVD
    print("Application de la décomposition SVD...")
    U, S, Vt = np.linalg.svd(audio_data, full_matrices=False)
    
    # Garder les k premières composantes
    S_k = np.diag(S[:k])
    U_k = U[:, :k]
    Vt_k = Vt[:k, :]
    
    # Reconstruire le signal compressé
    compressed_audio_data = np.dot(U_k, np.dot(S_k, Vt_k))
    
    # Si les données sont toujours 2D, les aplatir
    if compressed_audio_data.ndim > 1:
        compressed_audio_data = compressed_audio_data.flatten()
    
    # Sauvegarder l'audio compressé dans un fichier temporaire WAV
    temp_output_file = "temp_compressed.wav"
    wav.write(temp_output_file, sample_rate, compressed_audio_data.astype(np.int16))
    
    # Convertir le fichier WAV en MP3 avec une qualité réduite
    print(f"Conversion du fichier compressé en MP3...")
    audio = AudioSegment.from_wav(temp_output_file)
    audio.export(output_audio_file, format="mp3", bitrate="128k")  # Vous pouvez ajuster le bitrate
    
    # Supprimer le fichier temporaire WAV
    os.remove(temp_output_file)
    
    # Comparer les tailles des fichiers
    original_size = os.path.getsize(input_audio_file)
    compressed_size = os.path.getsize(output_audio_file)
    print(f"Taille du fichier original : {original_size / 1024:.2f} Ko")
    print(f"Taille du fichier compressé (MP3) : {compressed_size / 1024:.2f} Ko")
    
    # Calculer l'erreur relative
    calculate_error(audio_data.flatten(), compressed_audio_data)

# Fonction pour calculer l'erreur relative
def calculate_error(original_audio, compressed_audio):
    error = np.linalg.norm(original_audio - compressed_audio) / np.linalg.norm(original_audio)
    print(f"Erreur relative après compression : {error:.4f}")
    if error > 0.1:
        print("L'erreur est significative, peut-être que la compression est trop forte.")
    else:
        print("La compression a bien réduit la taille sans perte importante.")

# Exemple d'utilisation
input_audio_file = "audio.mp3"  # Remplacez par votre fichier
output_audio_file = "compressed_audio.mp3"  # Fichier compressé en MP3
k = 50  # Niveau de compression
compress_audio_simple(input_audio_file, output_audio_file, k)


Conversion du fichier audio en mono...
Conversion des données audio en matrice 2D...
Application de la décomposition SVD...
Conversion du fichier compressé en MP3...
Taille du fichier original : 4248.46 Ko
Taille du fichier compressé (MP3) : 1417.54 Ko
Erreur relative après compression : 0.0000
La compression a bien réduit la taille sans perte importante.
